In [2]:
import torch
import warnings
from transformers import logging

from medixar.agent.agent import Agent

warnings.filterwarnings("ignore")
logging.set_verbosity_error()

In [3]:
agent = Agent(
    model="meta-llama/Llama-3.2-1B-Instruct",
    tools={},
    system_prompts_file="medixar/prompts/system_prompts.txt",
    system_prompt_type="MEDICAL_ASSISTANT",
    device="auto",
    torch_dtype=torch.float16
)

response = agent.generate("Hello, how are you?")
print(response)

I'm doing well, thank you for asking. How can I assist you today? Is there a specific medical topic you'd like to know more about, or do you have a personal health concern you'd like to discuss? I'm here to provide information and guidance to the best of my abilities.
